In [2]:
import numpy as np
import pandas as pd
import random
import copy
import matplotlib.pyplot as plt
import math
from sklearn.utils import shuffle
from tqdm import tqdm_notebook 

import cvxopt.solvers
import numpy.linalg as la
import logging

In [3]:
class Chromosome:
 
    def __init__(self,  vardim, bound):

        self.vardim = vardim
        self.bound = bound
        self.fitness = 0.
 
    def generate(self):

        len = self.vardim
        rnd1 = np.random.random(size=len)-0.5
        rnd2 = np.random.random(size=len)-0.5
        self.chrom = np.zeros(len)
        self.cig = np.zeros(len)
        for i in range(0, len):
            self.chrom[i] = self.bound[0, i] + \
                (self.bound[1, i] - self.bound[0, i]) * rnd1[i]
            self.cig[i] = self.bound[0, i] + \
                (self.bound[1, i] - self.bound[0, i]) * rnd2[i]
        #print("chorm: {}, ".format(self.chrom)+"cigmal: {}".format(self.cig))
    def calculateFitness(self,dataset):
        
        #print(self.chrom)
        self.fitness = SVMResult(
            self.vardim, self.chrom, self.bound, dataset)
        
    def print_(self):
        
        print(self.chrom)

In [17]:
class GA:
 
    def __init__(self, sizepop, vardim, bound, MAXGEN, params, k):
        
        self.remain = sizepop
        self.realsize = self.remain
        self.sizepop = self.remain*k
        self.MAXGEN = MAXGEN
        self.vardim = vardim
        self.bound = bound
        self.k = k
        self.population = []
        self.fitness = np.zeros((self.sizepop, 1))
        self.trace = np.zeros((self.MAXGEN, 2))
        self.params = params
        self.dataset = self.creat_dataset(pd.read_csv("diabetes.csv"))
        self.lr = 1
        
    def creat_dataset(self, df):
        
        df = shuffle(df)
        size = len(df)
        df['split'] = 0
        df.iloc[0:math.ceil(0.3*size),-1] = 'train'
        #df.iloc[math.ceil(0.7*size):math.ceil(0.85*size), -1] = 'test'
        df.iloc[math.ceil(0.7*size):size, -1] = 'val' #调高验证集比例

        return df
 
    def initialize(self):
        
        for i in range(0, self.remain):
            ind = Chromosome(self.vardim, self.bound)
            ind.generate()
            self.population = np.append(self.population, ind)
        self.population = np.array(self.population) 
        
    def evaluate(self):
        
        fitness = []
        for i in tqdm_notebook(range(0, self.realsize)):
            self.population[i].calculateFitness(self.dataset)
            #print("###/n")
            fitness.append(self.population[i].fitness)
        self.fitness = np.array(fitness)
        best_idx = np.argmax(self.fitness)
        self.best_score = self.fitness[best_idx]
        self.best = self.population[best_idx]
        
    def show(self):
        
        for i in range(0, self.realsize):
            #print('chorme {}: '.format(i))
            self.population[i].print_()
            print(self.population[i].fitness)
 
    def solve(self):
        
        self.t = 0
        self.initialize()
        self.evaluate()
        
        best = np.max(self.fitness)
        bestIndex = np.argmax(self.fitness)
        self.best = copy.deepcopy(self.population[bestIndex])
        self.avefitness = np.mean(self.fitness)
        self.trace[self.t, 0] = self.best.fitness
        self.trace[self.t, 1] = self.avefitness
        if self.best>self.ever_best:
            self.ever_best = self.best
        print("本代最好的染色体: {}".format(self.best.chrom))
        print("本代最高分: {}".format(self.best_score))
        #print("第%d代: 最高分: %f; 平均分%f" % (
            #self.t, self.trace[self.t, 0], self.trace[self.t, 1]))
        while (self.t < self.MAXGEN - 1):
            
            self.t += 1
            self.crossover()
            self.mutation()
            self.evaluate()
            self.selection()
            
            best = np.max(self.fitness)
            bestIndex = np.argmax(self.fitness)
            self.best = copy.deepcopy(self.population[bestIndex])
            self.avefitness = np.mean(self.fitness)
            self.trace[self.t, 0] = self.best.fitness
            self.trace[self.t, 1] = self.avefitness
            if self.best>self.ever_best:
                self.ever_best = self.best
            
            print("本代最好的染色体: {}".format(self.population[bestIndex].chrom))
            print("本代最高分: {}".format(self.population[bestIndex].fitness))
            print("历史最高分: {}".format(self.ever_best.fitness))
            
        print("Optimal function value is: %f; " %
              self.ever_best.fitness)
        print("Optimal solution is:")
        print(self.best.chrom)
        self.printResult()
 
    def selection(self):
        
        #硬选择
        #print("max: {}".format(np.argmax(self.fitness)))
        #print(self.fitness[-1])
        sort_idx = np.argsort(self.fitness, axis=0).reshape(1,-1)[0]  #????
        top_idx = sort_idx[len(self.fitness)-self.remain-1:-1]
        #print(sort_idx[0:self.remain])
        new_fitness = copy.deepcopy(self.fitness[top_idx])
        self.fitness = new_fitness
        #print(len(self.fitness))
        new = copy.deepcopy(self.population[top_idx]) #深复制
        self.population  = new
        self.realsize = len(self.population)
        #print(self.realsize)
    
    """
     def selection(self):
     
        #软选择
        #print("max: {}".format(np.argmax(self.fitness)))
        #print(self.fitness)
        sort_idx = np.argsort(self.fitness, axis=0).reshape(1,-1)[0]
        #print(sort_idx[0:self.remain])
        new = copy.deepcopy(self.population[sort_idx[0:self.remain]])
        self.population  = new
        self.realsize = len(self.population)
        #print(self.realsize)
    """
    
    def crossover(self):
        
        newpop = []
       # print(self.sizepop-self.remain)
        #print(len(self.population)) 
        #print(self.sizepop-self.remain-10)
        for i in range(self.sizepop-self.remain-10):
            idx1 = random.randint(0, self.remain - 1)
            idx2 = random.randint(0, self.remain - 1)
            while idx2 == idx1:
                idx2 = random.randint(0, self.remain - 1)
            new = copy.deepcopy(self.population[idx1])
            new.chrom += self.population[idx2].chrom
            new.chrom /= 2
            new.cig += self.population[idx2].cig
            new.cig /= 2
            self.population = np.append(self.population, new)
        #print(len(self.population))
        for i in range(9):
            new = Chromosome(self.vardim, self.bound)
            new.generate()
            self.population = np.append(self.population, new)
        self.realsize = len(self.population)
        #print(self.realsize)
        
    def mutation(self):
        
        newpop = []
        self.lr *= 0.9
        for i in range(0, self.sizepop-1):
            p = random.random()-0.5
            newpop.append(copy.deepcopy(self.population[i]))
            rand = random.random()-0.5
            for j in range(0, self.vardim):
                #s = 0
                #while newpop[i].chrom[j]<=self.bound[0][j] or newpop[i].chrom[j]>=self.bound[1][j]: #限制bound
                    
                #s+=1
                randn = random.random()-0.5
                flag = 1
                while newpop[i].cig[j]==0 or flag==1:
                    flag = 0
                    newpop[i].cig[j] = newpop[i].cig[j]*np.exp(rand+randn)

                newpop[i].chrom[j] = newpop[i].cig[j]+newpop[i].cig[j]*randn*self.lr #增加学习率
        newpop.append(self.best)    #保留最佳    
                #print(s)
        self.population = np.array(newpop)
        self.realsize = len(self.population)
        #print(self.realsize)
 
    def printResult(self):
        
        x = np.arange(0, self.MAXGEN)
        y1 = self.trace[:, 0]
        y2 = self.trace[:, 1]
        plt.plot(x, y1, 'r', label='optimal value')
        plt.plot(x, y2, 'g', label='average value')
        plt.xlabel("Iteration")
        plt.ylabel("function value")
        plt.title("Genetic algorithm for function optimization")
        plt.legend()
        plt.show()


In [12]:
MIN_SUPPORT_VECTOR_MULTIPLIER = 1e-5

class SVM(object):
    def __init__(self, kernel, c):
        self._kernel = kernel
        self._c = c

    def train(self, X, y):
        
        lagrange_multipliers = self._compute_multipliers(X, y)
        return self._construct_predictor(X, y, lagrange_multipliers)

    def _gram_matrix(self, X):
        n_samples, n_features = X.shape
        K = np.zeros((n_samples, n_samples))
        
        for i, x_i in enumerate(X):
            for j, x_j in enumerate(X):
                K[i, j] = self._kernel(x_i, x_j)
        return K

    def _construct_predictor(self, X, y, lagrange_multipliers):
        support_vector_indices = \
            lagrange_multipliers > MIN_SUPPORT_VECTOR_MULTIPLIER

        support_multipliers = lagrange_multipliers[support_vector_indices]
        support_vectors = X[support_vector_indices]
        support_vector_labels = y[support_vector_indices]
        
        bias = np.mean(
            [y_k - SVMPredictor(
                kernel=self._kernel,
                bias=0.0,
                weights=support_multipliers,
                support_vectors=support_vectors,
                support_vector_labels=support_vector_labels).predict(x_k)
             for (y_k, x_k) in zip(support_vector_labels, support_vectors)])

        return SVMPredictor(
            kernel=self._kernel,
            bias=bias,
            weights=support_multipliers,
            support_vectors=support_vectors,
            support_vector_labels=support_vector_labels)

    def _compute_multipliers(self, X, y):
        n_samples, n_features = X.shape

        K = self._gram_matrix(X)
        
        P = cvxopt.matrix(np.outer(y, y) * K)
        q = cvxopt.matrix(-1 * np.ones(n_samples))
        
        G_std = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h_std = cvxopt.matrix(np.zeros(n_samples))

        G_slack = cvxopt.matrix(np.diag(np.ones(n_samples)))
        h_slack = cvxopt.matrix(np.ones(n_samples) * self._c)

        G = cvxopt.matrix(np.vstack((G_std, G_slack)))
        h = cvxopt.matrix(np.vstack((h_std, h_slack)))
        A = cvxopt.matrix(np.double(y), (1, n_samples))
        b = cvxopt.matrix(0.0)
        
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)

        return np.ravel(solution['x'])


class SVMPredictor(object):
    def __init__(self,
                 kernel,
                 bias,
                 weights,
                 support_vectors,
                 support_vector_labels):
        self._kernel = kernel
        self._bias = bias
        self._weights = weights         #shape = (sv_num, )
        self._support_vectors = support_vectors       #shape = (sv_num, feature, )
        self._support_vector_labels = support_vector_labels    #shape = (sv_num, )
        self._s = np.array([alpha*y for alpha,y in zip(self._weights,self._support_vector_labels)]) #shape = (sv_num, )

    def predict(self, x):
        
        result = self._bias
        for z_i, x_i, y_i in zip(self._weights,
                                 self._support_vectors,
                                 self._support_vector_labels):
            result += z_i * y_i * self._kernel(x_i, x)
        return np.sign(result).item()
    
    def predict_vec(self, X):
        """
        X.shape = (batch, feature)
        """
        
        k = np.array([[self._kernel(x_i, X_j) for x_i in self._support_vectors] for X_j in X])  #shape = (batch, sv_num, )
        score = np.dot(k, self._s.reshape(-1,1))  #shape = (batch, )
        return np.sign(score.reshape(-1,1))
            
        

In [13]:
class Kernel(object):
    @staticmethod
    def linear():
        return lambda x, y: np.inner(x, y)

    @staticmethod
    def gaussian(sigma):
        return lambda x, y: \
            np.exp(-np.sqrt(la.norm(x-y) ** 2 / (2 * sigma ** 2)))

    @staticmethod
    def _polykernel(dimension, offset):
        return lambda x, y: (offset + np.inner(x, y)) ** dimension

    @classmethod
    def inhomogenous_polynomial(cls, dimension):
        return cls._polykernel(dimension=dimension, offset=1.0)

    @classmethod
    def homogenous_polynomial(cls, dimension):
        return cls._polykernel(dimension=dimension, offset=0.0)

    @staticmethod
    def hyperbolic_tangent(kappa, c):
        return lambda x, y: np.tanh(kappa * np.dot(x, y) + c)

    @staticmethod
    def radial_basis(gamma=10):
        return lambda x, y: np.exp(-gamma*la.norm(np.subtract(x, y)))

In [14]:
def score(y_bar, val_y):
    
    Error = 0
    for i in range(len(y_bar)):
        miss=abs(y_bar[i-1]-val_y[i-1])
        Error += miss
        
    return Error / len(y_bar)
 
def SVMResult(vardim, x, bound, dataset):
    
    X = dataset.loc[dataset['split'] == 'train'].iloc[:,0:-2].values
    y = dataset.loc[dataset['split'] == 'train'].iloc[:,-2].values
    val_X = dataset.loc[dataset['split'] == 'val'].iloc[:,0:-2].values
    val_y = dataset.loc[dataset['split'] == 'val'].iloc[:,-2].values
    c = x[0]
    g = x[1]
    #f = x[2]#四参数
    rbf_kernel = Kernel.radial_basis(gamma=abs(g))
    svm = SVM(rbf_kernel, abs(c))
    predictor = svm.train(X, y)
    y_bar = predictor.predict_vec(val_X)
    #predictval=clf.predict(val_X)
    #print(predictval)
    #return msefunc(predictval,val_y) 
    return score(y_bar, val_y)


In [15]:
bound = np.array([[0,0],[20, 1]])
ga = GA(50, 2, bound, 50, [0.5, 0.1, 0.5], 7)#调高种群大小， 调低生存比 //又调回来了
ga.solve()

W:\conda\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


     pcost       dcost       gap    pres   dres
 0: -7.9409e+02 -9.6856e+03  1e+04  1e-01  2e-15
 1: -8.8767e+02 -1.5565e+03  7e+02  7e-03  1e-15
 2: -1.4309e+03 -1.4805e+03  5e+01  3e-04  1e-15
 3: -1.4361e+03 -1.4366e+03  5e-01  3e-06  1e-15
 4: -1.4362e+03 -1.4362e+03  5e-03  3e-08  1e-15
 5: -1.4362e+03 -1.4362e+03  5e-05  3e-10  1e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5869e+02 -4.2399e+02  1e+03  1e+00  3e-16
 1: -1.3664e+02 -2.4313e+02  1e+02  1e-02  3e-16
 2: -1.6250e+02 -1.6682e+02  4e+00  5e-04  4e-16
 3: -1.6535e+02 -1.6539e+02  4e-02  5e-06  4e-16
 4: -1.6538e+02 -1.6538e+02  4e-04  5e-08  3e-16
 5: -1.6538e+02 -1.6538e+02  4e-06  5e-10  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.5591e+02 -4.0682e+02  1e+03  1e+00  4e-16
 1: -1.3240e+02 -2.3647e+02  1e+02  1e-02  3e-16
 2: -1.5700e+02 -1.6125e+02  4e+00  5e-04  5e-16
 3: -1.5979e+02 -1.5983e+02  4e-02  5e-06  2e-16
 4: -1.5982e+02 -1.5982e

KeyboardInterrupt: 